<a href="https://colab.research.google.com/github/tempiatine/PySpark-tasks/blob/main/Books_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, max, sum, avg, month, count

# Создание SparkSession
spark = SparkSession.builder.appName("Read CSV Meteo").getOrCreate()

# Чтение CSV-файлов
books_df = spark.read.csv("/content/sample_data/books.csv", header=True, inferSchema=True)
authors_df = spark.read.csv("/content/sample_data/authors.csv", header=True, inferSchema=True)

# Регистрация DataFrame как временные таблицы
books_df.createOrReplaceTempView("books")
authors_df.createOrReplaceTempView("authors")

# Выполнение JOIN-запроса с использованием SQL по топ-5 авторам
books_authors_df = spark.sql("""
SELECT *
FROM books b
JOIN authors a
ON b.author_id = a.author_id
""")

# Выполнение SQL-запроса c топ-5 авторами по выручке
top_authors = books_authors_df.select("name", "price").groupBy('name').agg(sum('price').alias('total_revenue')).orderBy(sum('price').desc()).limit(5)

# Выполнение SQL-запроса с подсчетом количества книг в каждом жанре
genre_books = books_authors_df.select("genre", "book_id").groupBy('genre').agg(count('book_id')).orderBy(count('book_id').desc())

# Выполнение SQL-запроса с расчетом среней цены книги у автора
avg_price = books_authors_df.select('name', 'price').groupBy('name').agg(avg('price')).orderBy(avg('price').desc())

# Выполнение SQL-запроса с сортировкой по цене книг, выпущенных после 2000 года
price_xxi = books_authors_df.select('title', 'publish_date', 'price').filter(col('publish_date') >= '2000-01-01').orderBy(col('price').desc())

# Показ результатов
# books_authors_df.show()
top_authors.show()
genre_books.show()
avg_price.show()
price_xxi.show()

# Остановка SparkSession
spark.stop()

+-------+-------+---------+-----------+-----+------------+---------+---------+----------+---------+
|book_id|  title|author_id|      genre|price|publish_date|author_id|     name|birth_date|  country|
+-------+-------+---------+-----------+-----+------------+---------+---------+----------+---------+
|      1| Book_1|        2|    Mystery|73.57|  1980-12-31|        2| Author_2|1965-12-31|   Canada|
|      2| Book_2|        1|Non-Fiction| 41.1|  1982-12-31|        1| Author_1|1960-12-31|    India|
|      3| Book_3|       10|    Fiction|10.63|  1984-12-31|       10|Author_10|2005-12-31|    India|
|      4| Book_4|        9|Non-Fiction|46.31|  1986-12-31|        9| Author_9|2000-12-31|Australia|
|      5| Book_5|        7|    Science|31.13|  1988-12-31|        7| Author_7|1990-12-31|      USA|
|      6| Book_6|        4|Non-Fiction| 83.7|  1990-12-31|        4| Author_4|1975-12-31|       UK|
|      7| Book_7|        6|Non-Fiction|40.36|  1992-12-31|        6| Author_6|1985-12-31|      USA|
